In [1]:
# Leaky integrator model of Echo State Network
import numpy as np
from matplotlib import pyplot as plt
% matplotlib inline

In [4]:
PPL_SIZE = 30
N_NODES = 155
INTERNAL_NODE = 5
N_EDGE = 2280
N_TEST = 20
SPECT_RADIUS = 0.99
MUTATION_GROWTH = 8
REWIRING_GROWTH = 15
MATING_GROWTH = 15
W_IN = (np.random.rand(N_NODES, 1) * 2 - 1)*0.1
W_IN[-INTERNAL_NODE:] = [0]

trainlen = 1200
future = 1000
buffer = 100

In [31]:
def correct_dimensions(s, targetlength):
    if s is not None:
        s = np.array(s)
        if s.ndim == 0:
            s = np.array([s] * targetlength)
        elif s.ndim == 1:
            if not len(s) == targetlength:
                raise ValueError("arg must have length " + str(targetlength))
        else:
            raise ValueError("Invalid argument")
    return s


def identity(x):
    return x


class LI_ESN_internal:

    def __init__(self, n_inputs, n_outputs, n_reservoir=200, W=None, W_in=None,
                 noise=0.001, input_shift=None,
                 input_scaling=None, feedback_scaling=None,
                 teacher_scaling=None, teacher_shift=None,
                 out_activation=identity, inverse_out_activation=identity,
                 internal_node=5,
                 random_state=None, time_scale=None):
        # check for proper dimensionality of all arguments and write them down.
        self.n_inputs = n_inputs
        self.n_reservoir = n_reservoir
        self.n_outputs = n_outputs
        self.noise = noise
        self.internal_node = internal_node
        self.input_shift = correct_dimensions(input_shift, n_inputs)
        self.input_scaling = correct_dimensions(input_scaling, n_inputs)

        self.teacher_scaling = teacher_scaling
        self.teacher_shift = teacher_shift

        self.out_activation = out_activation
        self.inverse_out_activation = inverse_out_activation
        self.random_state = random_state
        self.time_scale = time_scale
        self.W = W
        self.W_in = W_in

        # the given random_state might be either an actual RandomState object,
        # a seed or None (in which case we use numpy's builtin RandomState)
        if isinstance(random_state, np.random.RandomState):
            self.random_state_ = random_state
        elif random_state:
            try:
                self.random_state_ = np.random.RandomState(random_state)
            except TypeError as e:
                raise Exception("Invalid seed: " + str(e))
        else:
            self.random_state_ = np.random.mtrand._rand

    def _update(self, state, input_pattern):
        # leaky integrator model:
        # it can adjust timescales for each neurons.
        preactivation = (np.dot(self.W, state) + np.dot(self.W_in, input_pattern))
        state = (1 - self.time_scale) * state + self.time_scale * np.tanh(preactivation)
        return (state + self.noise * self.time_scale * (self.random_state_.rand(self.n_reservoir) - 0.5))

    def calc_lyapunov_exp(self, inputs, initial_distance, n):
        if inputs.ndim < 2:
            inputs = np.reshape(inputs, (len(inputs), -1))
        states1 = np.zeros((inputs.shape[0], self.n_reservoir))
        states2 = np.zeros((inputs.shape[0], self.n_reservoir))
        transient = min(int(inputs.shape[0] / 10), 100)
        for i in range(1, transient):
            states1[i, :] = self._update(states1[i-1], inputs[i, :])
        states2[transient-1, :] = states1[transient-1, :]
        states2[transient-1, n] = states2[transient-1, n] + initial_distance
        gamma_k_list = []
        for k in range(transient, inputs.shape[0]):
            states1[k, :] = self._update(states1[k-1], inputs[k, :])
            states2[k, :] = self._update(states2[k-1], inputs[k, :])
            gamma_k = np.linalg.norm(states2[k, :]-states1[k, :])
            gamma_k_list.append(gamma_k/initial_distance)
            states2[k, :] = states1[k, :] + (initial_distance/gamma_k)*(states2[k, :]-states1[k, :])
        lyapunov_exp = np.mean(np.log(gamma_k_list))
        return lyapunov_exp
            
    
    def fit(self, inputs, outputs):
        if inputs.ndim < 2:
            inputs = np.reshape(inputs, (len(inputs), -1))
        if outputs.ndim < 2:
            outputs = np.reshape(outputs, (len(outputs), -1))
        inputs_scaled = inputs
        teachers_scaled = outputs

        # step the reservoir through the given input,output pairs:
        states = np.zeros((inputs.shape[0], self.n_reservoir))
        for n in range(1, inputs.shape[0]):
            states[n, :] = self._update(states[n - 1], inputs_scaled[n, :])
        transient = min(int(inputs.shape[0] / 10), 100)
        
        self.W_out = np.dot(np.linalg.pinv(states[transient:, :-self.internal_node]),teachers_scaled[transient:, :]).T

        # remember the last state for later:
        self.laststate = states[-1, :]
        self.lastinput = inputs[-1, :]
        self.lastoutput = teachers_scaled[-1, :]
            
        # apply learned weights to the collected states:
        pred_train = np.dot(states[:, :-self.internal_node], self.W_out.T)
        return pred_train

    def predict(self, inputs, continuation=True):
        if inputs.ndim < 2:
            inputs = np.reshape(inputs, (len(inputs), -1))
        n_samples = inputs.shape[0]

        if continuation:
            laststate = self.laststate
            lastinput = self.lastinput
            lastoutput = self.lastoutput
        else:
            laststate = np.zeros(self.n_reservoir)
            lastinput = np.zeros(self.n_inputs)
            lastoutput = np.zeros(self.n_outputs)

        inputs = np.vstack([lastinput, inputs])
        states = np.vstack(
            [laststate, np.zeros((n_samples, self.n_reservoir))])
        outputs = np.vstack(
            [lastoutput, np.zeros((n_samples, self.n_outputs))])

        for n in range(n_samples):
            states[n + 1, :] = self._update(states[n, :], inputs[n + 1, :])
            outputs[n + 1, :] = np.dot(self.W_out,states[n + 1, :-self.internal_node])

        return self.out_activation(outputs[1:])


In [32]:
def make_data_for_narma(length):
    tau = 0.01
    buffer = 100
    x = np.random.rand(length+100)*0.5
    y = np.zeros(length)
    for i in range(length):
        if i < 29:
            y[i] = 0.2*y[i-1] + 0.004*y[i-1]*np.sum(np.hstack((y[i-29:], y[:i]))) + 1.5*x[i-29+100]*x[i+100] + 0.001
        else:
            y[i] = 0.2*y[i-1] + 0.004*y[i-1]*np.sum(np.hstack((y[i-29:i]))) + 1.5*x[i-29+100]*x[i+100] + 0.001
    return x, y

In [33]:
def make_init_ppl(spectral_radius):
    population = []
    for i in range(PPL_SIZE):
        W = np.ones(N_NODES**2)*(10/N_NODES)
        tmp = np.random.choice(N_NODES**2, N_EDGE, replace=False)
        mask = [False if i in tmp else True for i in range(N_NODES**2)]
        mask = np.array(mask)
        W[mask] = 0
        W = W.reshape(N_NODES, N_NODES)
        # radius = np.max(np.abs(np.linalg.eigvals(W)))
        # W = W * (spectral_radius / radius)
        population.append([W, 0.0])
    return population

In [34]:
time_scale = np.ones(N_NODES)
def generation(time_scale, population, n_ppl=PPL_SIZE, n_test=N_TEST):
    data_pool = []
    for i in range(len(population)):
        esn = LI_ESN_internal(n_inputs=1,
                              W=population[i][0],
                              W_in=W_IN,
                              n_outputs=1,
                              n_reservoir=N_NODES,
                              noise=0,
                              internal_node=INTERNAL_NODE,
                              time_scale=time_scale)
        fitness_list = []
        for k in range(n_test):
            data, target = make_data_for_narma(trainlen+future)
            pred_training = esn.fit(data[buffer:buffer+trainlen], target[:trainlen])
            prediction = esn.predict(data[trainlen+buffer:])
            fitness = np.sqrt(np.mean((np.reshape(prediction, -1)-np.reshape(target[trainlen:], -1))**2)/np.var(target[trainlen:]))
            fitness_list.append(fitness)
        # print(np.mean(fitness_list))
        population[i][1] = np.mean(fitness_list)
    return population

In [35]:
def mutation(population, n_growth=MUTATION_GROWTH, n_ppl=PPL_SIZE):
    for i in np.random.choice(n_ppl, n_growth, replace=False):
        tmp_W = population[i][0].reshape(N_NODES**2)
        while True:
            mutation_index = np.random.choice(N_NODES**2)
            if abs(tmp_W[mutation_index]) > 0:
                break
        dist_range = 0.1*abs(tmp_W[mutation_index])
        tmp_W[mutation_index] = tmp_W[mutation_index] + dist_range*(np.random.rand()*2-1)
        population.append([tmp_W.reshape((N_NODES, N_NODES)), 0.0])
    return population

In [36]:
def rewiring(population, n_growth=REWIRING_GROWTH, n_ppl=PPL_SIZE):
    for i in np.random.choice(n_ppl, n_growth, replace=False):
        tmp_W = population[i][0].reshape(N_NODES**2)
        while True:
            rewiring_index = np.random.choice(N_NODES**2)
            if abs(tmp_W[rewiring_index]) > 0:
                break
        if np.random.rand() > 0.5:
            j = rewiring_index % N_NODES
            new_rewiring_index = np.random.choice(N_NODES)*N_NODES+j
        else:
            i = rewiring_index // N_NODES
            new_rewiring_index = i*N_NODES + np.random.choice(N_NODES)
        tmp_saved = tmp_W[new_rewiring_index]
        tmp_W[new_rewiring_index] = tmp_W[rewiring_index]
        tmp_W[rewiring_index] = tmp_saved
        population.append([tmp_W.reshape((N_NODES, N_NODES)), 0.0])
    return population

In [37]:
def mating(population, n_growth=MATING_GROWTH, n_ppl=PPL_SIZE):
    for n in range(n_growth):
        i, j = np.random.choice(n_ppl, 2, replace=False)
        overlap = []
        oneside_tuple = []
        parent1 = population[i][0].reshape(N_NODES**2)
        parent2 = population[j][0].reshape(N_NODES**2)
        for k in range(N_NODES**2):
            if parent1[k]*parent2[k] != 0:
                overlap.append(k)
            elif parent1[k] != 0 or parent2[k] != 0:
                oneside_tuple.append((1 if parent1[k] != 0 else 2, k))
        new_W = np.zeros(N_NODES**2)
        if len(overlap) > 0:
            for edge_index in overlap:
                new_W[edge_index] = parent1[edge_index] if np.random.rand() > 0.5 else parent2[edge_index]
        selected_index = np.random.choice(len(oneside_tuple), N_EDGE-len(overlap), replace=False)
        for edge_index in selected_index:
            new_W[oneside_tuple[edge_index][1]] = parent1[oneside_tuple[edge_index][1]] if oneside_tuple[edge_index][0]==1 else parent2[oneside_tuple[edge_index][1]]
        population.append([new_W.reshape((N_NODES, N_NODES)), 0.0])
    return population

In [38]:
def natural_selection(population, n_ppl=PPL_SIZE):
    fitness_list = [population[i][1] for i in range(len(population))]
    argsort = np.argsort(fitness_list)
    population = np.array(population)
    sorted_population = population[argsort, :]
    population = sorted_population[:n_ppl]
    print('best_fitness:', sorted_population[0][1], 'average_fitness:', np.mean([population[i][1] for i in range(n_ppl)]))
    return population

In [ ]:
init_population = make_init_ppl(SPECT_RADIUS)
for generation_idx in range(100):
    new_population = mating(init_population)
    new_population = rewiring(new_population)
    # new_population = mutation(new_population)
    new_population = generation(time_scale, new_population)
    new_population = natural_selection(new_population)

best_fitness: 0.4179409278432722 average_fitness: 0.43451697241890186
best_fitness: 0.4168933370125855 average_fitness: 0.43107816624688045
best_fitness: 0.41661353148424213 average_fitness: 0.42714179875284836
best_fitness: 0.41674302361979876 average_fitness: 0.4260541969942374
best_fitness: 0.41526932381557885 average_fitness: 0.42372983317943785
best_fitness: 0.4126826263410511 average_fitness: 0.42283766800997735
best_fitness: 0.40814964355859784 average_fitness: 0.42111803205126497
best_fitness: 0.4092222841092057 average_fitness: 0.42048016990023934
best_fitness: 0.4102740258164871 average_fitness: 0.42005788018504403
best_fitness: 0.41261002671141866 average_fitness: 0.41990270349917663
best_fitness: 0.4112519405950029 average_fitness: 0.4191216158136634
best_fitness: 0.41379779873381173 average_fitness: 0.41885889589607106
best_fitness: 0.41140348042355396 average_fitness: 0.41789796273848895
best_fitness: 0.41268564488494314 average_fitness: 0.41769681260002495
best_fitness: 

In [ ]:
# speciationして構造を保護したほうがいいかもしれない！！！
# タスクの選定は大事。J_ijの違いがパフォーマンスに強く影響を与える設定が望ましい。
# noiseを加えるのは考えてみてもいいかも。
# sparsityを変えてみるとか。
# W_inも最適化させた方がいい気がする。